In [3]:
import requests
from time import sleep
import pandas as pd
import unicodedata
import string
import urllib
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
#③実行する
import gspread
import json
#ServiceAccountCredentials：Googleの各サービスへアクセスできるservice変数を生成します。
from oauth2client.service_account import ServiceAccountCredentials 
from gspread_dataframe import get_as_dataframe, set_with_dataframe
#2つのAPIを記述しないとリフレッシュトークンを3600秒毎に発行し続けなければならない
import ssl
from datetime import datetime as dt
ssl._create_default_https_context = ssl._create_unverified_context
# from utils import *
import datetime
import time
import os

In [4]:
report_df = pd.read_csv(r'csv/report1685968221958.csv')
#提案区分を削除
report_df = report_df.drop('提案区分', axis=1)
#欠損値を削除
report_df = report_df.dropna()
report_df

,イベント日,店舗名／法人名,都道府県,一般並び(スロ),並び人数(スロ)
0,2023/01/01,アスカ杉戸店,埼玉県,49.0,48.0
1,2023/01/01,一番舘草加店,埼玉県,74.0,152.0
2,2023/01/01,アリーナ平塚店,埼玉県,27.0,12.0
3,2023/01/01,ジアスセンター南,神奈川県,23.0,198.0
4,2023/01/01,オータ川島店,埼玉県,25.0,169.0
...,...,...,...,...,...
2251,2023/05/29,スーパーBOSS,栃木県,21.0,35.0
2252,2023/05/29,KING BOSS 1000,栃木県,25.0,33.0
2253,2023/05/30,第1新効,群馬県,8.0,9.0
2254,2023/05/30,イーグル袖ヶ浦店,千葉県,40.0,64.0


In [ ]:
#店舗名の表記ゆれを修正するために一旦グーグルで検索して店舗名を取得する
anaslo_scraping_error_tenpo_name_list = []
browser = webdriver.Chrome(ChromeDriverManager().install())
serched_df = pd.DataFrame(index=[], columns=[])
slopachi_and_anaslo_tenpo_name_dict = {}
#no_scraping_anaslo_tenpo_name = []
for slopachi_tenpo_name in sorted(list(sammy_sf_report_df['店舗名／法人名'].unique())):
    url = f"https://www.google.com/search?q=アナスロ  {slopachi_tenpo_name} データ一覧&hl=ja&source=hp/"
    print(url)
    #print('データ取得中...30秒ほど時間がかかります。')
    browser.get(url)
    html = browser.page_source.encode('utf-8')
    elems_rec = browser.find_elements_by_class_name('MjjYud')
    elems_rec[0].text.split('\n')
    print('アナスロと出る部分',elems_rec[0].text.split('\n')[1])

    scraping_anaslo_name = elems_rec[0].text.split('\n')[0]
    slopachi_and_anaslo_tenpo_name_dict[slopachi_tenpo_name] = scraping_anaslo_name

    time.sleep(2)